In [3]:
import numpy as np
import pandas as pd
from scipy.special import softmax

In [7]:
df=pd.read_csv('chatbot_dataset.csv')
df.head(2)

,User ID,User Utterance,Bot Response,Timestamp,Context/Session ID,Entities,User Feedback,Conversation Outcome,User Profile,Channel/Platform,Language,User Emotion/Sentiment,Location,User Segment
0,uKqYhMMQ5S,Charge bar between follow student.,Important law into large example range. Player...,2023-10-31 18:02:06,4dfe56dc-efe2-49c6-be9f-ce5b84ca4de4,event,negative,incomplete,Annette Henderson,social media,German,confused,Sydney,returning customers
1,YOonrpgxp9,Bad every reflect huge contain.,Policy argue agree character go recent. When r...,2023-10-31 18:02:06,74c97616-9df6-460d-991d-c6f6f6ae5354,location,neutral,specific outcome,Nicholas Haney,mobile app,Chinese,frustrated,London,returning customers


In [11]:
x=df['User Utterance']
y=df['Bot Response']

In [15]:
y[0]

'Important law into large example range. Player seem force with partner sometimes happen southern.'

In [16]:
x[0]

'Charge bar between follow student.'

In [18]:
vocab=["My name is Bablu","Bablu goes to college"]

In [19]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(vocab)

In [20]:
sequences=tokenizer.texts_to_sequences(vocab)

In [21]:
sequences

[[2, 3, 4, 1], [1, 5, 6, 7]]

In [22]:
tokenizer.word_index

{'bablu': 1, 'my': 2, 'name': 3, 'is': 4, 'goes': 5, 'to': 6, 'college': 7}

In [4]:
# 4 dimensions for each word
import math
def positionalEncodings(sequence):
    
    PE=[]
    d_model=4
    for pos in range(sequence):
        EncodingsOfEach=[]
        for i in range(d_model):
            if i%2==0:
                temp=pos/(math.pow(10000,(2*i)/d_model))
                EncodingsOfEach.append(math.sin(temp))
            else:
                temp=pos/(math.pow(10000,(2*(i-1))/d_model))
                EncodingsOfEach.append(math.cos(temp))
        PE.append(EncodingsOfEach)
    return PE

In [5]:
tokens = ["The", "cat", "sat"]
PE=(positionalEncodings(len(tokens)))


In [6]:
a=[1,0,1,0]
b=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
result=np.matmul(a,b)
result

array([1, 0, 1, 0])

In [7]:
a=np.array([1,0,1,0])
b=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
result=np.matmul(a,b)
result
print(a.shape)
print(b.shape)
print(result.shape)


(4,)
(4, 4)
(4,)


In [8]:
#input will be the embedding(+PE) 
# eg:[[1,0,1,0],[0,1,0,1],[1,1,1,1]]
def selfAttention(embeddings,W_q, W_k,W_v):
    numberOfWords=len(embeddings)
    Q_K_V_ofAllWords=[]
    for i in range(numberOfWords):
        Q_of_given_word=np.matmul(embeddings[i],W_q)
        K_of_given_word=np.matmul(embeddings[i],W_k)
        V_of_given_word=np.matmul(embeddings[i],W_v)
        temp=[]
        temp.append(Q_of_given_word)
        temp.append(K_of_given_word)
        temp.append(V_of_given_word)
        Q_K_V_ofAllWords.append(temp)
    d_k=math.sqrt(len(Q_K_V_ofAllWords[0][1]))
    scoreOfAllWords=[]
    for i in range(numberOfWords):
        query=Q_K_V_ofAllWords[i][0]
        scoreOfEachWord=[]
        for j in range(numberOfWords):
            key=Q_K_V_ofAllWords[j][1]
            scoreOfEachWord.append(np.matmul(query,key)/d_k)
        scoreOfAllWords.append(scoreOfEachWord)
    attentionWeightsOfAllWords=[]
    for i in range(numberOfWords):
        attentionWeightsOfAllWords.append(softmax(scoreOfAllWords[i]))
    
    finalAttentionOutput=[]
    for i in range(numberOfWords):
        d_v = len(Q_K_V_ofAllWords[0][2])
        temp = np.zeros(d_v)
        for j in range(numberOfWords):
            temp += attentionWeightsOfAllWords[i][j] * Q_K_V_ofAllWords[j][2]
        finalAttentionOutput.append(temp)
    return finalAttentionOutput

    
    


In [20]:
temp_matrix=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
embeddings=np.array([[1,0,1,0],[0,1,0,1],[1,1,1,1]])
x=(selfAttention(embeddings,temp_matrix,temp_matrix,temp_matrix))

In [31]:
x

[array([0.8446376, 0.5776812, 0.8446376, 0.5776812]),
 array([0.5776812, 0.8446376, 0.5776812, 0.8446376]),
 array([0.78805844, 0.78805844, 0.78805844, 0.78805844])]

In [ ]:
temp=x+embeddings


numpy.ndarray

In [34]:
def Add_and_normalize(output_of_previous,input_of_previous):
    added=output_of_previous+input_of_previous
    output=[]
    eps = 1e-6
    for i in range(len(added)):
        mean = np.mean(added[i])
        std_dev = np.std(added[i])
        normalized_array=(added[i]-mean)/(std_dev+eps)
        output.append(normalized_array)
    return output

    

In [40]:
temp1=(Add_and_normalize(x,embeddings))